# Task
Build, train, and evaluate a spam classification model using the `enron_spam_data.csv` dataset, including data preprocessing, TF-IDF feature extraction, and evaluation with metrics like accuracy, precision, recall, F1-score, and a confusion matrix.

In [ ]:
import pandas as pd

# SOURCE: https://www.kaggle.com/datasets/marcelwiechmann/enron-spam-data?select=enron_spam_data.csv

# Construct the file path using the gdrive_extracted_data_path variable
file_path = "enron_spam_data.csv"

# Load the CSV file into a pandas DataFrame
df = pd.read_csv(file_path)

# Display the first 5 rows of the DataFrame
print("First 5 rows of the DataFrame:")
print(df.head())

# Print a concise summary of the DataFrame
print("\nDataFrame Info:")
df.info()

First 5 rows of the DataFrame:
   Message ID                       Subject  \
0           0  christmas tree farm pictures   
1           1      vastar resources , inc .   
2           2  calpine daily gas nomination   
3           3                    re : issue   
4           4     meter 7268 nov allocation   

                                             Message Spam/Ham        Date  
0                                                NaN      ham  1999-12-10  
1  gary , production from the high island larger ...      ham  1999-12-13  
2             - calpine daily gas nomination 1 . doc      ham  1999-12-14  
3  fyi - see note below - already done .\nstella\...      ham  1999-12-14  
4  fyi .\n- - - - - - - - - - - - - - - - - - - -...      ham  1999-12-14  

DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33716 entries, 0 to 33715
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Message ID  33716 no

## Data Preprocessing and Exploration

Perform initial exploration of the dataset, clean and preprocess the text data for model training.


First, We need to check for missing values in the DataFrame to understand the data completeness. Then, we will drop the 'Message ID' column as it's not relevant for classification.



In [ ]:
print("Missing values before cleaning:")
print(df.isnull().sum())

# Drop the 'Message ID' column
df = df.drop('Message ID', axis=1)

print("\nDataFrame after dropping 'Message ID' column:")
print(df.head())

Missing values before cleaning:
Message ID      0
Subject       289
Message       371
Spam/Ham        0
Date            0
dtype: int64

DataFrame after dropping 'Message ID' column:
                        Subject  \
0  christmas tree farm pictures   
1      vastar resources , inc .   
2  calpine daily gas nomination   
3                    re : issue   
4     meter 7268 nov allocation   

                                             Message Spam/Ham        Date  
0                                                NaN      ham  1999-12-10  
1  gary , production from the high island larger ...      ham  1999-12-13  
2             - calpine daily gas nomination 1 . doc      ham  1999-12-14  
3  fyi - see note below - already done .\nstella\...      ham  1999-12-14  
4  fyi .\n- - - - - - - - - - - - - - - - - - - -...      ham  1999-12-14  


we will fill missing values in 'Subject' and 'Message' columns with empty strings, create a combined 'text' column from 'Subject' and 'Message', and then convert the 'Spam/Ham' column into numerical labels ('label')



In [ ]:
df['Subject'] = df['Subject'].fillna('')
df['Message'] = df['Message'].fillna('')

# Create a new column 'text' by concatenating 'Subject' and 'Message'
df['text'] = df['Subject'] + ' ' + df['Message']

# Convert 'Spam/Ham' to numerical labels
df['label'] = df['Spam/Ham'].map({'ham': 0, 'spam': 1})

print("DataFrame after filling missing values, creating 'text' and 'label' columns:")
print(df[['text', 'label']].head())

DataFrame after filling missing values, creating 'text' and 'label' columns:
                                                text  label
0                      christmas tree farm pictures       0
1  vastar resources , inc . gary , production fro...      0
2  calpine daily gas nomination - calpine daily g...      0
3  re : issue fyi - see note below - already done...      0
4  meter 7268 nov allocation fyi .\n- - - - - - -...      0



To preprocess the text data we need to import necessary NLTK modules, download the required 'stopwords', 'wordnet', and 'omw-1.4' corpora, and then define a text cleaning function that performs lowercasing, punctuation removal, digit removal, tokenization, stop word removal, and lemmatization. Finally, we will apply this function to the 'text' column.



In [ ]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

# Download necessary NLTK data
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt')
nltk.download('punkt_tab') # Added this download

# Initialize lemmatizer and stopwords
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()
    # Remove numbers
    text = re.sub(r'\d+', '', text) # [Corrected, digit, removal ,to, handle ,multiple ,digits]
    # Remove punctuation
    text = re.sub(r'[^\w\s]', '', text)

    # Tokenize
    tokens = word_tokenize(text)
    # Remove stop words and lemmatize
    cleaned_tokens = []
    for token in tokens:
        if token not in stop_words:
            cleaned_tokens.append(lemmatizer.lemmatize(token))
    # Join back into string
    return ' '.join(cleaned_tokens)

# Apply the preprocessing function to the 'text' column
df['text'] = df['text'].apply(preprocess_text)

print("DataFrame after applying text preprocessing:")
print(df[['text', 'label']].head())

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


DataFrame after applying text preprocessing:
                                                text  label
0                        christmas tree farm picture      0
1  vastar resource inc gary production high islan...      0
2  calpine daily gas nomination calpine daily gas...      0
3  issue fyi see note already done stella forward...      0
4  meter nov allocation fyi forwarded lauri allen...      0


# Task
Convert the preprocessed text data into numerical features using TF-IDF Vectorizer.

## Feature Extraction (TF-IDF)

Convert the preprocessed text data into numerical features using a TF-IDF Vectorizer, which is suitable for text classification.


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=5000) # Limiting to 5000 features for demonstration

# Apply TF-IDF to the preprocessed text data
tfidf_features = tfidf_vectorizer.fit_transform(df['text'])

print("Shape of TF-IDF features:", tfidf_features.shape)
print("Sample TF-IDF features (first 5 rows, first 10 columns):")
print(tfidf_features[:5, :10].toarray())

Shape of TF-IDF features: (33716, 5000)
Sample TF-IDF features (first 5 rows, first 10 columns):
[[0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.03592908 0.33767781 0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.03165009
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.07265102
  0.         0.03947017 0.         0.        ]]


## Split Data into Training and Testing Sets

Divide the dataset into training and testing subsets to train the model on one part and evaluate its performance on unseen data.


In [ ]:
from sklearn.model_selection import train_test_split

# Define features (X) and target (y)
X = tfidf_features
y = df['label']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Print the shapes of the resulting datasets
print("Shape of X_train:", X_train.shape)
print("Shape of X_test:", X_test.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of y_test:", y_test.shape)

Shape of X_train: (26972, 5000)
Shape of X_test: (6744, 5000)
Shape of y_train: (26972,)
Shape of y_test: (6744,)


## Train a Spam Classifier Model

Train a classification model, such as Multinomial Naive Bayes on the training data.


In [ ]:
from sklearn.naive_bayes import MultinomialNB

# Instantiate a Multinomial Naive Bayes classifier
model = MultinomialNB()

# Train the model on the training data
model.fit(X_train, y_train)

print("Multinomial Naive Bayes model trained successfully.")

Multinomial Naive Bayes model trained successfully.


## Evaluate Model Performance

Evaluate the trained model's performance using appropriate metrics like accuracy, precision, recall, and F1-score, and display a confusion matrix for detailed insights.


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# Make predictions on the test set
y_pred = model.predict(X_test)

# Calculate evaluation metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

# Print the metrics
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")

# Generate and print the confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("\nConfusion Matrix:")
print(conf_matrix)

Accuracy: 0.9862
Precision: 0.9807
Recall: 0.9924
F1-score: 0.9865

Confusion Matrix:
[[3242   67]
 [  26 3409]]


## Final Task

summary of the spam classification model's performance and any potential next steps or improvements.


## Summary:

### Q&A
The spam classification model demonstrates strong performance, with an accuracy of 0.9841, precision of 0.9809, recall of 0.9881, and an F1-score of 0.9845.

### Data Analysis Key Findings
*   Text data was successfully converted into numerical features using a TF-IDF Vectorizer, resulting in a feature matrix of shape (33716, 5000).
*   The dataset was split into training and testing sets with an 80/20 ratio:
    *   Training set: 26972 samples for both features and labels.
    *   Testing set: 6744 samples for both features and labels.
*   A Multinomial Naive Bayes classifier was trained, achieving high performance on the test set.
*   The model's performance metrics are:
    *   **Accuracy**: 0.9841
    *   **Precision**: 0.9809
    *   **Recall**: 0.9881
    *   **F1-score**: 0.9845
*   The confusion matrix indicates:
    *   3243 True Negatives (correctly identified non-spam)
    *   3394 True Positives (correctly identified spam)
    *   66 False Positives (non-spam incorrectly classified as spam)
    *   41 False Negatives (spam incorrectly classified as non-spam)


In [ ]:
import joblib
import os

# Define the file paths for saving
model_path = os.path.join("", 'spam_classifier_model.joblib')
vectorizer_path = os.path.join("",'tfidf_vectorizer.joblib')

# Save the trained model
joblib.dump(model, model_path)
print(f"Trained model saved to: {model_path}")

# Save the fitted TF-IDF vectorizer
joblib.dump(tfidf_vectorizer, vectorizer_path)
print(f"TF-IDF vectorizer saved to: {vectorizer_path}")

print("Both the model and vectorizer have been saved successfully to Google Drive.")

Trained model saved to: /content/gdrive/MyDrive/enron_spam_data_extracted/spam_classifier_model.joblib
TF-IDF vectorizer saved to: /content/gdrive/MyDrive/enron_spam_data_extracted/tfidf_vectorizer.joblib
Both the model and vectorizer have been saved successfully to Google Drive.


In [ ]:
from sklearn.linear_model import LogisticRegression

# Instantiate a Logistic Regression model
# Using max_iter to ensure convergence, especially with large datasets or default solver
logistic_model = LogisticRegression(max_iter=1000, random_state=42)

# Train the model on the training data
logistic_model.fit(X_train, y_train)

print("Logistic Regression model trained successfully.")

Logistic Regression model trained successfully.


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

# Re-define features (X) and target (y) as they were not found
X = tfidf_features
y = df['label']

# Re-split the data into training and testing sets to ensure X_train and y_train are defined
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Instantiate a Logistic Regression model
# Using max_iter to ensure convergence, especially with large datasets or default solver
logistic_model = LogisticRegression(max_iter=1000, random_state=42)

# Train the model on the training data
logistic_model.fit(X_train, y_train)

print("Logistic Regression model trained successfully.")

Logistic Regression model trained successfully.


# Task
Evaluate the Logistic Regression model's performance using accuracy, precision, recall, F1-score, and a confusion matrix on the test data (`X_test`, `y_test`).

## Evaluate Logistic Regression Model

Evaluate the Logistic Regression model's performance using accuracy, precision, recall, F1-score, and a confusion matrix on the test data (`X_test`, `y_test`).


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# Make predictions on the test set using the logistic regression model
y_pred_logistic = logistic_model.predict(X_test)

# Calculate evaluation metrics for the Logistic Regression model
accuracy_logistic = accuracy_score(y_test, y_pred_logistic)
precision_logistic = precision_score(y_test, y_pred_logistic)
recall_logistic = recall_score(y_test, y_pred_logistic)
f1_logistic = f1_score(y_test, y_pred_logistic)

# Print the metrics
print(f"Logistic Regression Model Performance:")
print(f"Accuracy: {accuracy_logistic:.4f}")
print(f"Precision: {precision_logistic:.4f}")
print(f"Recall: {recall_logistic:.4f}")
print(f"F1-score: {f1_logistic:.4f}")

# Generate and print the confusion matrix
conf_matrix_logistic = confusion_matrix(y_test, y_pred_logistic)
print("\nConfusion Matrix (Logistic Regression):")
print(conf_matrix_logistic)

Logistic Regression Model Performance:
Accuracy: 0.9889
Precision: 0.9833
Recall: 0.9951
F1-score: 0.9891

Confusion Matrix (Logistic Regression):
[[3251   58]
 [  17 3418]]


## Train Random Forest Classifier

Train a Random Forest Classifier model on the preprocessed training data (`X_train`, `y_train`) for spam classification.


In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Instantiate a RandomForestClassifier model
# Set n_estimators to 100 and random_state for reproducibility
random_forest_model = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the model on the training data
random_forest_model.fit(X_train, y_train)

print("Random Forest Classifier model trained successfully.")

Random Forest Classifier model trained successfully.


## Evaluate Random Forest Classifier

Evaluate the Random Forest Classifier model's performance using accuracy, precision, recall, F1-score, and a confusion matrix on the test data (`X_test`, `y_test`).


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# Make predictions on the test set using the random forest model
y_pred_rf = random_forest_model.predict(X_test)

# Calculate evaluation metrics for the Random Forest Classifier model
accuracy_rf = accuracy_score(y_test, y_pred_rf)
precision_rf = precision_score(y_test, y_pred_rf)
recall_rf = recall_score(y_test, y_pred_rf)
f1_rf = f1_score(y_test, y_pred_rf)

# Print the metrics
print(f"Random Forest Classifier Model Performance:")
print(f"Accuracy: {accuracy_rf:.4f}")
print(f"Precision: {precision_rf:.4f}")
print(f"Recall: {recall_rf:.4f}")
print(f"F1-score: {f1_rf:.4f}")

# Generate and print the confusion matrix
conf_matrix_rf = confusion_matrix(y_test, y_pred_rf)
print("\nConfusion Matrix (Random Forest Classifier):")
print(conf_matrix_rf)

Random Forest Classifier Model Performance:
Accuracy: 0.9867
Precision: 0.9802
Recall: 0.9939
F1-score: 0.9870

Confusion Matrix (Random Forest Classifier):
[[3240   69]
 [  21 3414]]


## Summary:

### Data Analysis Key Findings

*   **Logistic Regression Model Performance**:
    *   Achieved high performance metrics: Accuracy of 0.9889, Precision of 0.9833, Recall of 0.9951, and an F1-score of 0.9891.
    *   The confusion matrix showed 3251 True Negatives, 58 False Positives, 17 False Negatives, and 3418 True Positives, indicating a very low rate of misclassifying spam as non-spam (false negatives).
*   **Random Forest Classifier Model Performance**:
    *   Also demonstrated strong performance: Accuracy of 0.9867, Precision of 0.9802, Recall of 0.9939, and an F1-score of 0.9870.
    *   The confusion matrix reported 3240 True Negatives, 69 False Positives, 21 False Negatives, and 3414 True Positives, similarly showing a low number of false negatives.



In [ ]:
import joblib
import os

# Define the file path for saving the logistic regression model
logistic_model_path = os.path.join("", 'logistic_regression_model.joblib')

# Save the trained logistic regression model
joblib.dump(logistic_model, logistic_model_path)
print(f"Logistic Regression model saved to: {logistic_model_path}")

Logistic Regression model saved to: /content/gdrive/MyDrive/enron_spam_data_extracted/logistic_regression_model.joblib
